## Demo

In [1]:
import time
import random as rd
import pandas as pd
import numpy as np
from tqdm import tqdm

import requests # 페이지를 요청하는 기능 
from bs4 import BeautifulSoup # 파이썬으로 HTML을 다루는 기능
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
headers = {
    "User-Agent": user_agent
}

In [2]:
def setRestaurantPages(url: str):
    url = url   
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 200번대 코드가 아니면 코드가 멈춤
    print("Response successful!")
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    restraurant_pages = []
    total_restraurants = int(soup.find_all('span', class_="b")[1].text)
    main_pages = total_restraurants//30 if total_restraurants%30==0 else total_restraurants//30+1
    for i in tqdm(range(main_pages-1), desc="Set restaurant pages"):
        if i > 0:
            current_url = url[:48] + f'oa{i*30}' + url[47:]
            # print(f'current_url: {current_url}')
            response = requests.get(current_url, headers=headers)
            response.raise_for_status()
            html = response.text
            soup = BeautifulSoup(html, "html.parser")

        for restraurant_page in soup.find_all('div', class_="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"):
            restraurant_pages.append("https://www.tripadvisor.com/" + restraurant_page.find('a')['href'])
            # print(restraurant_page.find('a')['href'])
        time.sleep
        # print(f'Total restraurant pages: {len(restraurant_pages)}')
        
    print(f'Total restraurant pages: {len(restraurant_pages)}')
    
    return restraurant_pages

In [3]:
def createDataset():
    dataset = pd.DataFrame({
        "name": [],
        "category": [],
        "description": [],
        "time": [],
        "URL": [],
        "priceLow": [],
        "priceHigh": [],
        "reviews": [],
        "adress": [],
        "rating": []
    })
    
    return dataset

In [9]:
def setWebdriver():
    options = Options()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    # options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    # options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    # options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    return driver

In [10]:
def buildDataset(dataset, pages: list):
    cnt = 0
    for restaurant_page in tqdm(pages, desc="Restaurants"):
        response = requests.get(restaurant_page, headers=headers)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")

        # url
        url = restaurant_page;    print(f'URL: {url}')
        
        # Using selenium
        driver.get(url)

        #  time
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'NehmB')))
            time.sleep(3)
            element = driver.find_element(By.CLASS_NAME, 'NehmB')
            element.click()

            time_list = [] 
            WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "RiEuX.f")))
            time_elements = driver.find_elements(By.CLASS_NAME, "RiEuX.f")
            for time_element in time_elements:
                time_list.append(time_element.text.replace('\n', ':'))
        except:
            time_list = np.NaN
        print(f'Times: {time_list}')

        # description
        try:
            WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.LINK_TEXT, "View all details")))
            element = driver.find_element(By.LINK_TEXT, "View all details")
            element.click()
            WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.TocEc._Z.S2.H2._f > div > div > div.kwVln > div > div:nth-child(1) > div > div.jmnaM")))
            element = driver.find_element(By.CSS_SELECTOR, "#BODY_BLOCK_JQUERY_REFLOW > div.VZmgo.D.X0.X1.Za > div > div.TocEc._Z.S2.H2._f > div > div > div.kwVln > div > div:nth-child(1) > div > div.jmnaM")
            description = element.text
        except:
            description = np.NaN
        print(f'Description: {description}\n')

        # name 
        try:
            name = soup.find('h1', class_="HjBfq").text;    print(f'Name: {name}')
        except:
            name = np.NaN

        # category
        category = ''
        for x in soup.find('span', class_="DsyBj DxyfE"):
            if '$' not in x.text:
                category += f'{x.text}, '
        category = category[:-2]
        print(f'Category: {category}')

        # adress
        adress = soup.find_all('a', class_="AYHFM")[1].text;   print(f'Adress: {adress}')

        # rating
        try:
            rating = soup.find('span', class_="ZDEqb").text;    print(f'Rating: {rating}')
        # rating = soup.select_one("#component_52 > div.hILIJ > div > div:nth-child(1) > div > div:nth-child(1) > div.QEQvp > span.ZDEqb"); print(f'Rating: {rating}')
        except:
            rating = np.NaN

        # price
        try:
            priceLow = soup.find('div', class_="SrqKb").text.split(' - ')[0] 
            priceHigh = soup.find('div', class_="SrqKb").text.split(' - ')[1]
        except:
            priceLow = np.NaN; priceHigh = np.NaN
        print (f'PriceLow: {priceLow}, PriceHigh: {priceHigh}')

        # reviews, only english reviews
        total_reviews = int(soup.find_all('span', class_="count")[0].text[1:-1].replace(',', ''));    print(f'Total_reviews: {total_reviews}')
        review_pages = total_reviews//15 if total_reviews%15 == 0 else total_reviews//15 + 1;    print(f'Review pages: {review_pages}')
        review_list = []
        for i in tqdm(range(0, review_pages), desc='Review crwaling...'):
            if i > 1:
                url_list = url.split('-Reviews-')
                current_url = url_list[0] + f'-Reviews-or{15*i}-' + url_list[1]
                # print(f'current_url: {current_url}')
                response = requests.get(current_url, headers=headers)
                response.raise_for_status()
                html = response.text
                soup = BeautifulSoup(html, "html.parser") 

            reviews = soup.find_all('p', class_="partial_entry");
            for review in reviews:
                review_list.append(review.text)
            #     print(review.text)
            # print(f'Reviews: {len(review_list)}')

            time.sleep(rd.uniform(0.1, 0.5))
        print(f'Reviews: {len(review_list)}')


        
        time.sleep(rd.uniform(0.5, 1.5))

        dataset.loc[cnt] = [name, category, description, time_list, url, priceLow, priceHigh, review_list, adress, rating]
        cnt += 1
        
        
    return dataset 

In [12]:
url = "https://www.tripadvisor.com/Restaurants-g298085-Da_Nang.html"
restraurant_pages = setRestaurantPages(url)

# create dataset
dataset = createDataset()

# set webdriver
driver = setWebdriver()

# start crawling
dataset = buildDataset(dataset, restraurant_pages[:10])

# save dataset
dataset.to_csv('test.csv')
print(f'Dataset build complete!\n')
dataset

Response successful!


Set restaurant pages: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


Total restraurant pages: 1811


C:\Users\user\AppData\Local\Temp\ipykernel_26272\939712835.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
Restaurants:   0%|          | 0/10 [00:00<?, ?it/s]

URL: https://www.tripadvisor.com//Restaurant_Review-g25231262-d24161498-Reviews-Co_Ba_Ph_Bo-Hai_Chau_Da_Nang.html
Times: ['Sun:10:00 AM - 10:00 PM', 'Mon:10:00 AM - 10:00 PM', 'Tue:10:00 AM - 10:00 PM', 'Wed:10:00 AM - 10:00 PM', 'Thu:10:00 AM - 10:00 PM', 'Fri:10:00 AM - 10:00 PM', 'Sat:10:00 AM - 10:00 PM']
Description: nan

Name: Cô Ba Phở Bò
Category: Seafood, Asian, Vietnamese
Adress: 154 Bạch Đằng Hải Châu 1, Hai Chau, Da Nang 550000 Vietnam
Rating: 5.0 
PriceLow: nan, PriceHigh: nan
Total_reviews: 44
Review pages: 3


...

Review crwaling...: 100%|██████████| 9/9 [00:11<00:00,  1.29s/it]


Reviews: 132


Restaurants: 100%|██████████| 10/10 [06:56<00:00, 41.65s/it]

Dataset build complete!



,name,category,description,time,URL,priceLow,priceHigh,reviews,adress,rating
0,Cô Ba Phở Bò,"Seafood, Asian, Vietnamese",NaN,"[Sun:10:00 AM - 10:00 PM, Mon:10:00 AM - 10:00...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,[Nice view and environment! \nFinally tasted t...,"154 Bạch Đằng Hải Châu 1, Hai Chau, Da Nang 55...",5.0
1,Cardi Pizzeria Bach Dang,"American, Steakhouse, Brew Pub",Premium Steak & Pizza Certified Imported Steak...,"[Sun:12:00 AM - 11:59 PM, Mon:12:00 AM - 11:59...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,[I came in for pizza because they look like th...,"124 Bạch Đằng, Hải Châu, Đà Nẵng, Da Nang Vietnam",5.0
2,Ca Chuon Co Vietnamese - Seafood Restaurant.,"Cafe, International, Asian","Ca Chuon Co, ('flying fish') a modern stylish ...","[Sun:06:00 AM - 10:00 PM, Mon:06:00 AM - 10:00...",https://www.tripadvisor.com//Restaurant_Review...,"₩12,953","₩38,860",[I was looking for some decent seafoods restau...,99 Vo Nguyen Giap Street Ngu Hanh Son District...,5.0
3,Bistecca Restaurant Danang,"Italian, Southwestern, European",Bistecca – Italian restaurant offering the ori...,"[Sun:06:00 AM - 10:00 PM, Mon:06:00 AM - 10:00...",https://www.tripadvisor.com//Restaurant_Review...,"₩3,886","₩64,767",[Beautiful dinner and great service! All meals...,"20 Dong Da 7F New Orient Hotel, Thuan Phuoc Wa...",5.0
4,3 Big - Nướng & Lẩu,"Pizza, Barbecue, Korean",3 BiG - Nướng & Lẩu Nhà hàng BBQ - Thịt nướng ...,"[Sun:11:00 AM - 11:00 PM, Mon:11:00 AM - 10:00...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,[Great service. Good price and good food. The ...,"134 Phan Châu Trinh, Phước Ninh, Hải Châu, Đà ...",5.0
5,Thìa Gỗ Restaurant Da Nang,"Vietnamese, Vegetarian Friendly",NaN,"[Sun:10:00 AM - 10:00 PM, Mon:10:00 AM - 10:00...",https://www.tripadvisor.com//Restaurant_Review...,"₩2,124","₩26,554",[Quynh was our server and took great care of u...,"53 Phan Thuc Duyen, My An, Da Nang 50507 Vietnam",5.0
6,Six On Six Cafe,"American, Cafe, Fusion","Hi! We're a small, funky specialty cafe on the...","[Sun:07:30 AM - 9:30 PM, Mon:07:30 AM - 9:30 P...",https://www.tripadvisor.com//Restaurant_Review...,"₩2,591","₩6,477",[My favorite western breakfast restaurant in a...,"64 Bà Huyện Thanh Quan K14/11 Luu Quang Thuan,...",4.5
7,La Cabana - House of BBQ,"Steakhouse, Grill, Diner",La Cabana is a Cozy Argentinian Steakhouse in ...,"[Sun:4:00 PM - 11:00 PM, Mon:4:00 PM - 11:00 P...",https://www.tripadvisor.com//Restaurant_Review...,"₩9,067","₩38,860",[Booked table after reading reviews and was no...,"505 D. Tran Hung Dao An Hai Trung, Son Tra, Da...",5.0
8,The Golden Dragon,"Chinese, International, Asian",Tease your taste bud with authentic Cantonese ...,"[Sun:11:30 AM - 2:00 PM:5:30 PM - 9:30 PM, Mon...",https://www.tripadvisor.com//Restaurant_Review...,NaN,NaN,[Employee is inattentive. Very bad experiences...,"2nd Floor, Grand Mercure Danang Hotel, Lot A1,...",5.0
9,The Gypsy Rooftop Kitchen & Bar,"Bar, European",THE GYPSY is perched on the rooftop of the sty...,"[Sun:5:00 PM - 12:00 AM, Mon:5:00 PM - 12:00 A...",https://www.tripadvisor.com//Restaurant_Review...,"₩15,933","₩69,041",[If you are looking for a great place for the ...,"20 Dong Da 10th floor, New Orient Hotel, Da Na...",5.0


---
### Partial testing

In [16]:
# rating

url = "https://www.tripadvisor.com//Restaurant_Review-g298085-d24082246-Reviews-3_Big_Nu_ng_L_u-Da_Nang.html"
response = requests.get(url, headers=headers)
response.raise_for_status()
html = response.text
soup = BeautifulSoup(html, "html.parser")
with open("output1.html", "w") as file:
    file.write(str(soup))   

# rating
rating = soup.find('span', class_="ZDEqb").text;    print(f'Rating: {rating}')
# rating = soup.select_one("#component_52 > div.hILIJ > div > div:nth-child(1) > div > div:nth-child(1) > div.QEQvp > span.ZDEqb"); 
# rating = soup.find('div', class_="QEQvp").text
print(f'Rating: {rating}')

Rating: 5.0 
Rating: 5.0 


In [11]:
# reviews, only english reviews

url = "https://www.tripadvisor.com//Restaurant_Review-g298085-d24082246-Reviews-3_Big_Nu_ng_L_u-Da_Nang.html"
response = requests.get(url, headers=headers)
response.raise_for_status()
html = response.text
soup = BeautifulSoup(html, "html.parser")

total_reviews = int(soup.find_all('span', class_="count")[0].text[1:-1].replace(',', ''));    print(f'Total_reviews: {total_reviews}')
review_pages = total_reviews//15 if total_reviews%15 == 0 else total_reviews//15 + 1;    print(f'Review pages: {review_pages}')
review_list = []
for i in tqdm(range(0, review_pages), desc='Review crwaling...'):
    if i > 1:
        url_list = url.split('-Reviews-')
        current_url = url_list[0] + f'-Reviews-or{15*i}-' + url_list[1]
        # print(f'current_url: {current_url}')
        response = requests.get(current_url, headers=headers)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser") 

    reviews = soup.find_all('p', class_="partial_entry");
    for review in reviews:
        review_list.append(review.text)
    #     print(review.text)
    # print(f'Reviews: {len(review_list)}')

    time.sleep(rd.uniform(0.1, 0.5))
print(f'Reviews: {len(review_list)}')

Total_reviews: 13
Review pages: 1


Review crwaling...: 100%|██████████| 1/1 [00:00<00:00,  2.37it/s]

Reviews: 13


In [13]:
# time test

driver = setWebdriver()
url = "https://www.tripadvisor.com/Restaurant_Review-g15296807-d13810289-Reviews-Thia_G_Restaurant_Da_Nang-My_An_Da_Nang.html"
response = driver.get(url)

try:
    WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#component_50 > div > div:nth-child(3) > span.DsyBj.YTODE > div > span.mMkhr")))
    element = driver.find_element(By.CSS_SELECTOR, "#component_50 > div > div:nth-child(3) > span.DsyBj.YTODE > div > span.mMkhr")
    time.sleep(1)
    element.click() 
except:
    times = np.NaN

time_list = [] 
time_elements = driver.find_elements(By.CLASS_NAME, "RiEuX.f")
for time_element in time_elements:
    time_list.append(time_element.text.replace('\n', ':'))
    print(time_element)
print(time_list)

C:\Users\user\AppData\Local\Temp\ipykernel_20692\4230172117.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", element="67F080C407A4187949C41A7E8080B652_element_75")>
<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", element="67F080C407A4187949C41A7E8080B652_element_76")>
<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", element="67F080C407A4187949C41A7E8080B652_element_77")>
<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", element="67F080C407A4187949C41A7E8080B652_element_78")>
<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", element="67F080C407A4187949C41A7E8080B652_element_79")>
<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", element="67F080C407A4187949C41A7E8080B652_element_80")>
<selenium.webdriver.remote.webelement.WebElement (session="256f007f9ab2cad63c076291897ee137", elemen